<a href="https://colab.research.google.com/github/aaliyah03/The-Spinal-Coders-85/blob/master/user_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# read csv into a dataframe
df_idf=pd.read_csv("https://raw.githubusercontent.com/vedangi2610/FHE-project-/master/data_preprocessing/preprocess_user.csv")

# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 Unnamed: 0         int64
id                 int64
username          object
created_at        object
lang              object
cleaned_tweets    object
favorite_count     int64
retweet_count      int64
text              object
tokens            object
lemmas            object
dtype: object
Number of questions,columns= (1200, 11)


In [10]:
import re
def pre_process(text):
    #print (text)
    text = ''.join([str(elem) for elem in text])
    #print (text)
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

#df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['prep_text'] = df_idf['lemmas'].apply(lambda x:pre_process(x))

#show the second 'text' just for fun
print (df_idf['prep_text'][2])
df_idf.head
df_idf.to_csv('preptext.csv')

 diplomat orf fellow rakesh sood rakeshnms collapse vishwaguru day ask h 


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
#stopwords=pd.read_csv("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")

#get the text column 
docs=df_idf['prep_text'].tolist()
print (docs)
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85)
word_count_vector=cv.fit_transform(docs)
list(cv.vocabulary_.keys())[:15]

[' count stellar reporter theprint cut rumour noise science s unique andhra covid ', ' houseful sign bengaluru crematorium run space cremate body rohiniswamy repo ', ' diplomat orf fellow rakesh sood rakeshnms collapse vishwaguru day ask h ', ' indias covid tragedy propaganda tool chinese state dead spare domain specialist ', ' wordedit rbis late decision ', ' hetero labs hyderabad seek nod tocilizumab key drug covid treatment india chandnahimani repor ', ' indias vaccination process slide disaster zone rapidly show sense urgency cr ', ' jolt bad press modi govt draft plan responsive hard work amrita nayak dutta ', ' wordedit pm modi call wb governor ', ' available protect matter ', ' pathan kasba shahjahanpur die covid symptom people begin ', ' iwitness mumbai slum cope covid case vaccination theprint sr correspondent angana chakr ', ' covid ', ' faheemyounus india cause collapse complacency mass gathering weak infrastructure icu beds ', ' covid death village follow official toll sile

['count',
 'stellar',
 'reporter',
 'theprint',
 'cut',
 'rumour',
 'noise',
 'science',
 'unique',
 'andhra',
 'covid',
 'houseful',
 'sign',
 'bengaluru',
 'crematorium']

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [34]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
#doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
print (len(sorted_items))
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items, 500)

# now print the results
print("\n=====Doc=====")
print(docs)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


12332

=====Doc=====
[' count stellar reporter theprint cut rumour noise science s unique andhra covid ', ' houseful sign bengaluru crematorium run space cremate body rohiniswamy repo ', ' diplomat orf fellow rakesh sood rakeshnms collapse vishwaguru day ask h ', ' indias covid tragedy propaganda tool chinese state dead spare domain specialist ', ' wordedit rbis late decision ', ' hetero labs hyderabad seek nod tocilizumab key drug covid treatment india chandnahimani repor ', ' indias vaccination process slide disaster zone rapidly show sense urgency cr ', ' jolt bad press modi govt draft plan responsive hard work amrita nayak dutta ', ' wordedit pm modi call wb governor ', ' available protect matter ', ' pathan kasba shahjahanpur die covid symptom people begin ', ' iwitness mumbai slum cope covid case vaccination theprint sr correspondent angana chakr ', ' covid ', ' faheemyounus india cause collapse complacency mass gathering weak infrastructure icu beds ', ' covid death village foll

In [27]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results